<a href="https://colab.research.google.com/github/kuheli31/Fake-News-Detector/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import re
import string
import requests
from requests.utils import quote
from urllib.parse import quote

# Enter the news to verify if its real or fake

In [23]:
news_check = input("Enter the news to verify if it is real or fake: ")

Enter the news to verify if it is real or fake: trump tarrifs on india


# Data Collection

**G-News API (*Real News*)**

In [29]:
from urllib.parse import quote
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Your existing code ---
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
endpoint = 'search'
question = input('Enter your question: ')
parameters = f'q={quote(question)}&lang=en&max=5'
news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'

response = requests.get(news_api_url)
data = response.json()

if 'articles' in data:
    articles_text = []  # to store article content for TF-IDF
    articles_info = []  # to store article details for CSV

    query_keywords = set(question.lower().split())

    for idx, article in enumerate(data['articles']):
        content = article['content'] if article['content'] else article['description']
        articles_text.append(content)

        # Keyword relevance %
        content_words = set(content.lower().split())
        relevant_words = query_keywords.intersection(content_words)
        keyword_relevance = (len(relevant_words) / len(query_keywords)) * 100 if query_keywords else 0

        articles_info.append({
            'Published Time': article['publishedAt'],
            'Title': article['title'],
            'Description': article['description'],
            'Content': content,
            'Image': article['image'],
            'Source Name': article['source']['name'],
            'Source URL': article['source']['url'],
            'Source Country': article['source']['country'],
            'URL': article['url'],
            'Keyword Relevance (%)': round(keyword_relevance, 2)
        })

    # --- TF-IDF & Cosine Similarity with query ---
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([question] + articles_text)  # include query as first row

    # Cosine similarity between query (0th) and each article
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Add similarity scores to articles info
    for i, score in enumerate(similarity_scores):
        articles_info[i]['Similarity with Query'] = round(score, 3)

    # Find most relevant article
    most_relevant_idx = similarity_scores.argmax()
    print(f"\nMost Relevant Article (Similarity {similarity_scores[most_relevant_idx]:.3f}):")
    print(f"Title: {articles_info[most_relevant_idx]['Title']}")
    print(f"Description: {articles_info[most_relevant_idx]['Description']}")
    print(f"URL: {articles_info[most_relevant_idx]['URL']}\n")

    # --- Save to CSV ---
    df = pd.DataFrame(articles_info)
    df = df.sort_values(by='Similarity with Query', ascending=False)  # sort by relevance
    df.to_csv('news_articles_with_similarity.csv', index=False)
    print("CSV saved as 'news_articles_with_similarity.csv'")

else:
    print(data)


Enter your question: india pakistan war

Most Relevant Article (Similarity 0.270):
Title: 'Old Practice of Pakistan To Sponsor Terror & Blame Neighbours': MEA Rejects 'Proxy War' Claim
Description: Earlier, Pakistan's Defence Minister Khawaja Asif accused India of having a hand in the military tensions between the Afghan Taliban and Pakistan and cast doubt on the 48-hour ceasefire with the Afghan government.
URL: https://www.republicworld.com/world-news/old-practice-of-pakistan-to-sponsor-terror-blame-neighbours-mea-rejects-proxy-war-claim

CSV saved as 'news_articles_with_similarity.csv'


**Twitter API**

In [30]:
# Importing necessary libraries for X (Twitter) API interaction
import tweepy

# Hardcoded Bearer Token
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

#Searching in tweets
question = news_check.strip()

# --- Main Execution Block ---
if not X_API_BEARER_TOKEN or not question:
    print("ERROR: Both the Bearer Token and the search query are required.")
else:
    try:
        # Initialize Tweepy Client
        client = tweepy.Client(X_API_BEARER_TOKEN)

        # API Call with image support
        response = client.search_recent_tweets(
            query=f'{question} -is:retweet has:images lang:en',
            max_results=10,
            tweet_fields=['created_at', 'public_metrics', 'attachments'],
            user_fields=['name', 'username', 'verified'],
            media_fields=['url', 'preview_image_url', 'type'],
            expansions=['author_id', 'attachments.media_keys']
        )

        if not response or not response.data:
            print("\nNo recent tweets found for that query.")
        else:
            tweets = response.data
            users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
            media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

            print("\n--- Found Tweets ---\n")
            for tweet in tweets:
                author = users.get(tweet.author_id)
                media_urls = []

                # Extract media URLs
                if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
                    for key in tweet.attachments['media_keys']:
                        if key in media and media[key].type == 'photo':
                            media_urls.append(media[key].url)

                tweet_url = f"https://x.com/{author.username}/status/{tweet.id}" if author else None
                author_url = f"https://x.com/{author.username}" if author else None

                print(f"Published Time: {tweet.created_at}")
                print(f"Tweet Text: {tweet.text}")
                if author:
                    print(f"Author Name: {author.name} (@{author.username})")
                    print(f"Author URL: {author_url}")
                print(f"Tweet URL: {tweet_url}")

                if media_urls:
                    print("Image URLs:")
                    for img in media_urls:
                        print(f"- {img}")
                else:
                    print("No images found.")

                print("\n")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---")
        print("The Bearer Token you provided is invalid. Please check your credentials on the X Developer Portal.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---")
        print("Your search query is invalid or too complex. Please check the format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")



An unexpected error occurred.
Error details: 429 Too Many Requests
Too Many Requests


**Reddit API**

In [6]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.5 MB/s eta 0:00:00


In [31]:
# Importing necessary libraries for Reddit API interaction
import praw
import datetime # To format the timestamp from Reddit

# --- API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- Search Parameters ---
try:
    question = news_check.strip()
    limit_str = input('How many results would you like? (e.g., 10): ').strip()
    num_results = int(limit_str) if limit_str.isdigit() else 10
except EOFError:
    print("\nNo input provided for search query. Exiting.")
    exit()

# --- Main Execution Block ---
if not REDDIT_CLIENT_ID or not REDDIT_CLIENT_SECRET or not question:
    print("ERROR: Client ID, Client Secret, and a search query are required.")
else:
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CLIENT_ID,
            client_secret=REDDIT_CLIENT_SECRET,
            user_agent=REDDIT_USER_AGENT,
        )

        search_results = reddit.subreddit("all").search(
            query=question,
            sort='new',
            limit=num_results
        )

        submissions = list(search_results)

        if not submissions:
            print("\nNo recent posts found for that query.")
        else:
            print(f"\n--- Found {len(submissions)} Posts ---\n")
            for submission in submissions:
                author_name = submission.author.name if submission.author else "[deleted]"
                post_url = f"https://www.reddit.com{submission.permalink}"
                author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"

                published_time = datetime.datetime.fromtimestamp(submission.created_utc)

                # --- Print basic info ---
                print(f"Published Time: {published_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"Post Title: {submission.title}")
                print(f"Subreddit: r/{submission.subreddit.display_name}")
                print(f"Score: {submission.score} (Upvote Ratio: {submission.upvote_ratio * 100:.0f}%)")
                print(f"Author: u/{author_name}")
                print(f"Author URL: {author_url}")
                print(f"Post URL: {post_url}")

                # --- Print full text if available ---
                if submission.selftext:
                    print(f"Post Text: {submission.selftext.replace(chr(10), ' ')}")

                # --- Extract image if any ---
                image_url = None
                if submission.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    image_url = submission.url
                elif hasattr(submission, 'preview'):
                    try:
                        image_url = submission.preview['images'][0]['source']['url']
                    except (KeyError, IndexError):
                        pass

                if image_url:
                    print(f"Image URL: {image_url}")
                else:
                    print("Image: None")

                print("\n" + "-"*60 + "\n")

    except praw.exceptions.PRAWException as e:
        print("\n--- API ERROR ---")
        print("An error occurred with the Reddit API. This could be due to invalid credentials or a connection problem.")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


How many results would you like? (e.g., 10): 10


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Found 10 Posts ---

Published Time: 2025-10-17 13:48:43
Post Title: [Business] - 'They forced me to do it': Trump says high tariffs on China 'not sustainable'; claims Beijing 'ripped off' America for years | Times of India
Subreddit: r/AutoNewspaper
Score: 1 (Upvote Ratio: 100%)
Author: u/AutoNewspaperAdmin
Author URL: https://www.reddit.com/user/AutoNewspaperAdmin
Post URL: https://www.reddit.com/r/AutoNewspaper/comments/1o91toa/business_they_forced_me_to_do_it_trump_says_high/
Image URL: https://external-preview.redd.it/DVilcudRw1IlSPuY9W-m-XMu3HMC1bHH8gZYE-tl6uw.jpeg?auto=webp&s=8021460ddb665f156cdff3cc8fd1058cb7bdd0c5

------------------------------------------------------------

Published Time: 2025-10-17 13:40:21
Post Title: The left doesn't care what Trump does or doesn't do.
Subreddit: r/AskThe_Donald
Score: 13 (Upvote Ratio: 73%)
Author: u/Pikachu_USA
Author URL: https://www.reddit.com/user/Pikachu_USA
Post URL: https://www.reddit.com/r/AskThe_Donald/comments/1o91mfk/the_

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-17 12:19:06
Post Title: 📰📰📰📰📰📰📰📰📰🆕🆕🆕🆕🆕🆕 Stock Market Morning Prep
Subreddit: r/Optionmillionaires
Score: 3 (Upvote Ratio: 100%)
Author: u/upbstock
Author URL: https://www.reddit.com/user/upbstock
Post URL: https://www.reddit.com/r/Optionmillionaires/comments/1o8zr2h/stock_market_morning_prep/
Post Text: New lawsuit: The U.S. Chamber of Commerce is suing the Trump administration over a $100,000 annual fee on H-1B visas, calling it unlawful and economically damaging to American businesses.  Investing and betting: CME plans to roll out financial contracts tied to both sports games and economic indicators, in a move similar to tie-ups like Intercontinental Exchange-Polymarket and Robinhood-Kalshi.  On the decline? "I'm more sanguine than my colleagues about inflation," Federal Reserve Governor Stephen Miran declared, because he expects "significant disinflation" in housing.  Sour loans As War

...

Image: None

------------------------------------------------------------

Published Time: 2025-10-17 10:28:41
Post Title: India contradicts Trump on Russian oil pledge
Subreddit: r/news2
Score: 1 (Upvote Ratio: 100%)
Author: u/donutloop
Author URL: https://www.reddit.com/user/donutloop
Post URL: https://www.reddit.com/r/news2/comments/1o8xoqf/india_contradicts_trump_on_russian_oil_pledge/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-17 10:28:30
Post Title: India contradicts Trump on Russian oil pledge
Subreddit: r/NoFilterNews
Score: 1 (Upvote Ratio: 100%)
Author: u/donutloop
Author URL: https://www.reddit.com/user/donutloop
Post URL: https://www.reddit.com/r/NoFilterNews/comments/1o8xomf/india_contradicts_trump_on_russian_oil_pledge/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-17 09:52:28
Post Title: India contradicts Trump on Russian oil pledge
Subreddit: r/energy
Score: 9 (Upvote Ratio: 100%)
Author: u/donutloop
Author URL: https://www.reddit.com/user/donutloop
Post URL: https://www.reddit.com/r/energy/comments/1o8x3b9/india_contradicts_trump_on_russian_oil_pledge/
Image: None

------------------------------------------------------------



# **Data Preprocessing**

In [8]:
!pip install emoji
!pip install beautifulsoup4 spacy scikit-learn
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


*Real news data in csv file*

In [11]:
!pip install rake_nltk

In [34]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import emoji
import re
import pandas as pd
from rake_nltk import Rake
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# --- GNews API Config ---
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
endpoint = 'search'

# --- Functions ---
def clean_text(text):
    if not text:
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

def extract_keywords_from_query(query, max_keywords=7):
    """Extract top keywords and ensure compatibility with past news searches."""
    r = Rake()
    r.extract_keywords_from_text(query)
    keywords = r.get_ranked_phrases()[:max_keywords]

    # Remove real-time trigger words to allow historical search
    forbidden_words = ['latest', 'breaking', 'new', 'recent', 'update', 'today', 'tonight']
    filtered_keywords = [kw for kw in keywords if all(fw not in kw.lower() for fw in forbidden_words)]

    # Add neutral word to force GNews historical search
    if filtered_keywords:
        filtered_keywords.append("report")

    return " ".join(filtered_keywords)

def fetch_news(query, max_results=5):
    parameters = f'q={quote(query)}&lang=en&max={max_results}'
    news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'
    response = requests.get(news_api_url)
    print("\n--- Fetching News ---")
    print("Request URL:", news_api_url)
    print("Status Code:", response.status_code)
    print("Raw Response (first 500 chars):", response.text[:500])
    return response.json().get('articles', [])

def preprocess_articles(articles):
    processed = []
    for article in articles:
        full_text = (article.get('title','') + " " +
                     article.get('description','') + " " +
                     article.get('content',''))
        processed.append({
            "published_at": article.get('publishedAt', ''),
            "title": clean_text(article.get('title', '')),
            "description": clean_text(article.get('description', '')),
            "content": clean_text(article.get('content', '')),
            "full_text": clean_text(full_text),
            "image": article.get('image', ''),
            "source_name": article['source'].get('name', ''),
            "source_url": article['source'].get('url', ''),
            "source_country": article['source'].get('country', ''),
            "article_url": article.get('url', '')
        })
    return pd.DataFrame(processed)

def calculate_relevance(df, query):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([query] + df['full_text'].tolist())
    similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
    df["relevance_score"] = similarities
    df = df.sort_values(by="relevance_score", ascending=False)
    return df

# --- Main ---
question = news_check  # User can enter a long paragraph, a few keywords, or request past news

# Step 1: Extract keywords and clean query
keyword_query = extract_keywords_from_query(question)
print("Keyword Query:", keyword_query)

# Step 2: Fetch articles
articles = fetch_news(keyword_query)

if not articles:
    print("⚠️ No articles found or API limit reached.")
else:
    df = preprocess_articles(articles)

    # Step 3: Rank by relevance to original query
    df = calculate_relevance(df, question)

    print(df[['title', 'relevance_score', 'article_url']])
    df.to_csv('preprocessed_news.csv', index=False)
    print("\nData saved to 'preprocessed_news.csv'.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Keyword Query: india pakistan war report

--- Fetching News ---
Request URL: https://gnews.io/api/v4/search?q=india%20pakistan%20war%20report&lang=en&max=5&apikey=d2b27a1e3f818564545508b6c744d24a
Status Code: 200
Raw Response (first 500 chars): {"information":{"realTimeArticles":{"message":"Real-time news data is only available on paid plans. Free plan has a 12-hour delay. Upgrade your plan here to remove the delay: https://gnews.io/change-plan"}},"articlesRemovedFromResponse":{"historicalArticles":{"message":"4 articles were removed from the response because historical data beyond 30 days is only available on paid plans. Upgrade your plan here to access historical data: https://gnews.io/change-plan"}},"totalArticles":18,"articles":[{"
                                               title  relevance_score  \
0  aakar patel | the new new india: kicked around...         0.102513   

                                         article_url  
0  https://www.deccanchronicle.com/opinion/column...

*Twitter datas in csv file*

In [42]:
# --- Import libraries ---
import pandas as pd
import re
import emoji
from bs4 import BeautifulSoup
import tweepy
import spacy

# --- Load SpaCy model for keyword extraction ---
nlp = spacy.load("en_core_web_sm")

# --- Your X API Bearer Token ---
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

# --- Functions ---

def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    return text.lower().strip()

def extract_keywords(sentence):
    """Extract important keywords from a full sentence."""
    doc = nlp(sentence)
    keywords = []
    for token in doc:
        if token.is_stop:
            continue
        if token.pos_ in ['NOUN', 'PROPN']:  # nouns and proper nouns
            keywords.append(token.text)
    return " ".join(keywords)

def fetch_tweets(query, max_results=20):
    """Fetch recent tweets using X API (Tweepy)."""
    client = tweepy.Client(X_API_BEARER_TOKEN)
    response = client.search_recent_tweets(
        query=f'{query} lang:en',  # only English tweets
        max_results=max_results,
        tweet_fields=['created_at', 'public_metrics', 'attachments'],
        user_fields=['name', 'username', 'verified'],
        media_fields=['url', 'preview_image_url', 'type'],
        expansions=['author_id', 'attachments.media_keys']
    )
    return response

def preprocess_tweets(response):
    """Preprocess tweets and return a DataFrame."""
    if not response or not response.data:
        return pd.DataFrame()  # empty dataframe if no tweets

    tweets = response.data
    users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
    media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

    processed = []
    for tweet in tweets:
        author = users.get(tweet.author_id)
        media_urls = []

        if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
            for key in tweet.attachments['media_keys']:
                if key in media and media[key].type == 'photo':
                    media_urls.append(media[key].url)

        processed.append({
            "published_at": tweet.created_at,
            "tweet_text": clean_text(tweet.text),
            "author_name": author.name if author else "",
            "author_username": author.username if author else "",
            "author_verified": author.verified if author else False,
            "tweet_url": f"https://x.com/{author.username}/status/{tweet.id}" if author else "",
            "author_url": f"https://x.com/{author.username}" if author else "",
            "image_urls": media_urls
        })

    return pd.DataFrame(processed)

# --- Main Execution ---

news_check = news_check.strip()

if not X_API_BEARER_TOKEN or not news_check:
    print("ERROR: Both the Bearer Token and the news sentence are required.")
else:
    try:
        # Step 1: Extract keywords
        keywords_query = extract_keywords(news_check)
        print(f"Searching tweets using keywords: {keywords_query}")

        # Step 2: Fetch tweets using keywords
        response = fetch_tweets(keywords_query)

        # Step 3: Preprocess and display tweets
        df = preprocess_tweets(response)
        if df.empty:
            print("\nNo recent tweets found for the given news keywords.")
        else:
            print(df)

            # Optional: Save preprocessed tweets for ML/NLP
            df.to_csv('preprocessed_tweets.csv', index=False)
            print("\nData saved to 'preprocessed_tweets.csv'.")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---\nCheck your Bearer Token.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---\nCheck your query format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


Searching tweets using keywords: india pakistan war

An unexpected error occurred: argument of type 'NoneType' is not iterable


In [39]:
# --- Import libraries ---
import pandas as pd
import re
import emoji
from bs4 import BeautifulSoup
import tweepy
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Load SpaCy model for keyword extraction ---
nlp = spacy.load("en_core_web_sm")

# --- Your X API Bearer Token ---
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

# --- Functions ---

def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    return text.lower().strip()

def extract_keywords_tfidf(text, top_n=15):
    """Extract keywords using TF-IDF from the news text."""
    # Clean text
    text_clean = clean_text(text)
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text_clean])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray()[0]
    # Pick top N keywords
    top_indices = scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[i] for i in top_indices]
    return top_keywords

def clean_keywords_for_query(keywords):
    """Clean keywords to avoid special character issues for X API search."""
    cleaned = [re.sub(r'[^A-Za-z0-9]', '', kw) for kw in keywords if kw.strip()]
    return [kw for kw in cleaned if kw]  # remove empty strings

def fetch_tweets(query, max_results=20):
    """Fetch recent tweets using X API (Tweepy)."""
    client = tweepy.Client(X_API_BEARER_TOKEN)
    response = client.search_recent_tweets(
        query=f'{query} lang:en',  # only English tweets
        max_results=max_results,
        tweet_fields=['created_at', 'public_metrics', 'attachments'],
        user_fields=['name', 'username', 'verified'],
        media_fields=['url', 'preview_image_url', 'type'],
        expansions=['author_id', 'attachments.media_keys']
    )
    return response

def preprocess_tweets(response, news_keywords):
    """Preprocess tweets and return a DataFrame with relevance score."""
    if not response or not response.data:
        return pd.DataFrame()  # empty dataframe if no tweets

    tweets = response.data
    users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
    media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

    processed = []
    for tweet in tweets:
        author = users.get(tweet.author_id)
        media_urls = []

        if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
            for key in tweet.attachments['media_keys']:
                if key in media and media[key].type == 'photo':
                    media_urls.append(media[key].url)

        tweet_text_clean = clean_text(tweet.text)

        # Compute relevance using TF-IDF cosine similarity
        combined_texts = [' '.join(news_keywords), tweet_text_clean]
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
        relevance = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        processed.append({
            "published_at": tweet.created_at,
            "tweet_text": tweet_text_clean,
            "author_name": author.name if author else "",
            "author_username": author.username if author else "",
            "author_verified": author.verified if author else False,
            "tweet_url": f"https://x.com/{author.username}/status/{tweet.id}" if author else "",
            "author_url": f"https://x.com/{author.username}" if author else "",
            "image_urls": media_urls,
            "relevance": relevance
        })

    # Sort by relevance
    df = pd.DataFrame(processed)
    df = df.sort_values(by='relevance', ascending=False).reset_index(drop=True)
    return df

# --- Main Execution ---


if not X_API_BEARER_TOKEN or not news_check:
    print("ERROR: Both the Bearer Token and the news sentence are required.")
else:
    try:
        # Step 1: Extract keywords using TF-IDF
        news_keywords = extract_keywords_tfidf(news_check, top_n=15)
        # Clean keywords for X search
        keywords_query = clean_keywords_for_query(news_keywords)
        query_string = ' '.join(keywords_query)
        print(f"Searching tweets using cleaned keywords: {query_string}")

        # Step 2: Fetch tweets using keywords
        response = fetch_tweets(query_string, max_results=50)

        # Step 3: Preprocess tweets and compute relevance
        df = preprocess_tweets(response, news_keywords)
        if df.empty:
            print("\nNo recent tweets found for the given news keywords.")
        else:
            print(df)

            # Optional: Save preprocessed tweets for ML/NLP
            df.to_csv('preprocessed_tweets.csv', index=False)
            print("\nData saved to 'preprocessed_tweets.csv'.")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---\nCheck your Bearer Token.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---\nCheck your query format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


Searching tweets using cleaned keywords: war pakistan india

An unexpected error occurred: 429 Too Many Requests
Too Many Requests


*Reddit's datas in csv file*

In [ ]:
!pip install praw
!pip install rake_nltk


In [33]:
import praw
import datetime
from bs4 import BeautifulSoup
import emoji
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Reddit API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- SpaCy NER ---
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    USE_SPACY = True
except:
    USE_SPACY = False
    print("⚠ spaCy model not found. Falling back to regex-based extraction.")

# ----------------- Utilities -----------------

def clean_text(text):
    if not text:
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def extract_keywords(text):
    if not text:
        return []
    keywords = set()
    # Numbers / Percentages
    keywords.update(re.findall(r'\b\d+(\.\d+)?%\b', text))
    keywords.update(re.findall(r'\b\d{1,6}\b', text))
    # Dates
    keywords.update(re.findall(r'\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\.?\s+\d{1,2},?\s*\d{0,4}\b', text, re.IGNORECASE))

    if USE_SPACY:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in ["PERSON","ORG","GPE","DATE","TIME","PERCENT","MONEY","QUANTITY"]:
                keywords.add(ent.text)
    else:
        # Fallback names
        name_matches = re.findall(r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b', text)
        keywords.update(name_matches)

    return list(set([kw.strip() for kw in keywords if len(kw) > 1]))

# ----------------- Reddit Fetch -----------------

def fetch_reddit_posts(query, max_posts=500):
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT
    )

    subreddit = reddit.subreddit("all")
    submissions = []

    for submission in subreddit.search(query=query, sort='new', limit=None):
        submissions.append(submission)
        if len(submissions) >= max_posts:
            break

    print(f"Fetched {len(submissions)} posts from Reddit.")
    return submissions

# ----------------- Relevance -----------------

def compute_relevance(news_keywords, post_text):
    combined_texts = [' '.join(news_keywords), post_text]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_texts)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# ----------------- Preprocess -----------------

def preprocess_reddit_posts(submissions, news_keywords):
    processed = []
    for submission in submissions:
        author_name = submission.author.name if submission.author else "[deleted]"
        post_url = f"https://www.reddit.com{submission.permalink}"
        author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"
        published_time = datetime.datetime.fromtimestamp(submission.created_utc)
        full_text = clean_text(submission.title + " " + submission.selftext)
        post_keywords = extract_keywords(full_text)
        relevance = compute_relevance(news_keywords, full_text)
        relevant_keywords = [kw for kw in post_keywords if kw.lower() in [k.lower() for k in news_keywords]]
        processed.append({
            "published_at": published_time.strftime('%Y-%m-%d %H:%M:%S'),
            "title": clean_text(submission.title),
            "selftext": clean_text(submission.selftext),
            "subreddit": submission.subreddit.display_name,
            "score": submission.score,
            "upvote_ratio": submission.upvote_ratio,
            "author_name": author_name,
            "author_url": author_url,
            "post_url": post_url,
            "keywords": relevant_keywords,
            "relevance": relevance
        })

    # Sort by relevance descending
    processed = sorted(processed, key=lambda x: x['relevance'], reverse=True)
    return pd.DataFrame(processed)

# ----------------- Main -----------------

news_text = news_check
news_keywords = extract_keywords(news_text)
query = ' '.join(news_keywords)

submissions = fetch_reddit_posts(query, max_posts=500)
df = preprocess_reddit_posts(submissions, news_keywords=news_keywords)

df.to_csv('relevant_reddit_posts.csv', index=False)
print(df)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Fetched 249 posts from Reddit.


/tmp/ipython-input-1595525740.py:29: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()


            published_at                                              title  \
0    2025-10-16 16:41:30  india stands firmly with afghanistan against p...   
1    2025-10-14 03:27:54  afghanistan-pakistan conflict – and why india ...   
2    2025-10-16 15:32:11  india says, "it stands with afghanistan agains...   
3    2025-10-16 21:20:02  india vs pakistan world cup game breaks women'...   
4    2025-10-14 13:00:35  pakistan defence minister khawaja asif sparked...   
..                   ...                                                ...   
244  2025-10-14 09:37:47             fifa world cup 2026 group stage leaked   
245  2025-10-14 08:40:43                                peaceful president…   
246  2025-10-14 07:04:56  from r/coolguides, australian medical educatio...   
247  2025-10-14 06:21:07                      wtc standings after ind vs wi   
248  2025-10-14 03:21:23  guys i think our neighbours still think they w...   

                                              selft

In [41]:
#Creating Dataframe
real_news = pd.read_csv('preprocessed_news.csv')
twitter_posts = pd.read_csv('preprocessed_tweets.csv')
reddit_posts = pd.read_csv('relevant_reddit_posts.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed_tweets.csv'

In [36]:
real_news

NameError: name 'real_news' is not defined

In [ ]:
twitter_posts

,published_at,tweet_text,author_name,author_username,author_verified,tweet_url,author_url,image_urls
0,2025-10-16 07:38:26+00:00,breaking: trump says modi told him that india ...,Ounka,OunkaOnX,False,https://x.com/OunkaOnX/status/1978727218252513742,https://x.com/OunkaOnX,['https://pbs.twimg.com/media/G3XZ6SZXkAA3UQa....
1,2025-10-16 07:38:21+00:00,pm modi's dedication to serving the nation is ...,Rekanjadhav,rekana24,False,https://x.com/rekana24/status/1978727197130252694,https://x.com/rekana24,['https://pbs.twimg.com/media/G3XZ4NSWcAAq0cQ....
2,2025-10-16 07:38:11+00:00,modi’s path to viksit bharat goes through its ...,Dhruvkishor Patil,DhruvkishorP,False,https://x.com/DhruvkishorP/status/197872715528...,https://x.com/DhruvkishorP,['https://pbs.twimg.com/media/G3XZ25PWkAEXfPi....
3,2025-10-16 07:38:01+00:00,"the story of two, bold and clear, mallanna’s v...",aakas,aakas27366,False,https://x.com/aakas27366/status/19787271147542...,https://x.com/aakas27366,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
4,2025-10-16 07:37:48+00:00,"mallanna stands with fearless tone, facing pow...",vikas,vikas26366,False,https://x.com/vikas26366/status/19787270594603...,https://x.com/vikas26366,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
5,2025-10-16 07:37:44+00:00,the modi government’s new wave of war talk aga...,MISHAL KHAN,MishalKhanNDU12,False,https://x.com/MishalKhanNDU12/status/197872704...,https://x.com/MishalKhanNDU12,['https://pbs.twimg.com/media/G3XZVDiWMAA6TtD....
6,2025-10-16 07:37:41+00:00,in response to u.s. president donald trump's c...,Peek TV,PeekTV_in,False,https://x.com/PeekTV_in/status/197872703068740...,https://x.com/PeekTV_in,['https://pbs.twimg.com/media/G3XZvruXAAAjgDk....
7,2025-10-16 07:37:35+00:00,@rapidresponse47 @potus modi to trump: https:/...,vadapav ugh,magnoninsig,False,https://x.com/magnoninsig/status/1978727006406...,https://x.com/magnoninsig,['https://pbs.twimg.com/media/G3XZuRfW4AA2Pax....
8,2025-10-16 07:37:34+00:00,"mallanna stands with fearless tone, facing pow...",varun,varunjaat7171,False,https://x.com/varunjaat7171/status/19787270020...,https://x.com/varunjaat7171,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
9,2025-10-16 07:37:23+00:00,guys do you know that our pm narendra modi is ...,Rintu Pawar,rintu_off,False,https://x.com/rintu_off/status/197872695404072...,https://x.com/rintu_off,['https://pbs.twimg.com/media/G3XZrbEXcAAd7RC....


In [ ]:
reddit_posts

,published_at,title,selftext,subreddit,score,upvote_ratio,author_name,author_url,post_url,image_url
0,2025-10-16 07:24:56,do you agree?,NaN,IndiaMemes,3,1.00,Calm_Dash250,https://www.reddit.com/user/Calm_Dash250,https://www.reddit.com/r/IndiaMemes/comments/1...,https://i.redd.it/1gnabsw1efvf1.jpeg
1,2025-10-16 07:17:53,us president donald trump claimed that prime m...,"speaking to reporters on october 15, trump sai...",BusinessTodayNews,1,1.00,BusinessToday,https://www.reddit.com/user/BusinessToday,https://www.reddit.com/r/BusinessTodayNews/com...,NaN
2,2025-10-16 07:16:09,qualcuno ha già parlato di equilibrio vita-lav...,NaN,LinkedInCringeIT,5,1.00,Mondonauta0206,https://www.reddit.com/user/Mondonauta0206,https://www.reddit.com/r/LinkedInCringeIT/comm...,https://i.redd.it/gyfmp98gcfvf1.jpeg
3,2025-10-16 07:02:26,new narendra modi upload: live: pm modi perfor...,this post contains content not supported on ol...,indian_discussions,1,1.00,yt-app,https://www.reddit.com/user/yt-app,https://www.reddit.com/r/indian_discussions/co...,https://external-preview.redd.it/nw6g1ihrhcMDV...
4,2025-10-16 06:46:53,trump says modi assured him india will halt ru...,NaN,u_Virtual_Swan_6551,1,1.00,Virtual_Swan_6551,https://www.reddit.com/user/Virtual_Swan_6551,https://www.reddit.com/r/u_Virtual_Swan_6551/c...,NaN
5,2025-10-16 06:27:43,[business] - trump says modi has agreed to sto...,NaN,AutoNewspaper,1,1.00,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,https://external-preview.redd.it/6hASWTjC-71TX...
6,2025-10-16 06:27:42,[top stories] - trump says modi has agreed to ...,NaN,AutoNewspaper,1,1.00,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,https://external-preview.redd.it/6hASWTjC-71TX...
7,2025-10-16 06:24:00,wait..... just a blank paper,NaN,IndiaMemes,3,1.00,Grumbling_Foot72,https://www.reddit.com/user/Grumbling_Foot72,https://www.reddit.com/r/IndiaMemes/comments/1...,https://i.redd.it/9a0kd5993fvf1.jpeg
8,2025-10-16 06:22:14,pawan kalyan grand welcome to pm narendra modi...,NaN,pawankalyan,4,1.00,Perfect_Sector1888,https://www.reddit.com/user/Perfect_Sector1888,https://www.reddit.com/r/pawankalyan/comments/...,https://i.redd.it/zsiru3713fvf1.jpeg
9,2025-10-16 06:17:34,perfect use of the song,NaN,delhi,2,0.55,kholeChature,https://www.reddit.com/user/kholeChature,https://www.reddit.com/r/delhi/comments/1o7yp8...,https://i.redd.it/mylyqv072fvf1.png
